In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
#import os
#mysql_jar_path = os.path.abspath("mysql-connector-j-9.0.0.jar")
#print("Caminho completo do JAR:", mysql_jar_path)


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn

2025-05-17 22:44:31.591201: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 22:44:31.826811: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-17 22:44:32.027607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747521872.119025    3073 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747521872.179897    3073 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747521872.384212    3073 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
import pandas as pd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, sum



In [5]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import yfinance as yf

In [4]:


ticker = "AAPL"
data = yf.download(ticker,
                   start="2018-01-01", 
                   end="2020-12-31")    

print(data.head())



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2018-01-02  40.426815  40.436204  39.722760  39.933979  102223600
2018-01-03  40.419788  40.964259  40.356426  40.490195  118071600
2018-01-04  40.607525  40.710787  40.384575  40.492528   89738400
2018-01-05  41.069866  41.156698  40.612231  40.703758   94640000
2018-01-08  40.917313  41.213014  40.818742  40.917313   82271200


In [5]:
csv_AAPL = f"{ticker}_AAPL_historical_train_data_spark.csv"
data.to_csv(csv_AAPL)

In [6]:
df_AAPL= pd.read_csv("AAPL_AAPL_historical_train_data_spark.csv")

In [7]:
df_AAPL.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,40.426815032958984,40.436204433265914,39.72276041223238,39.93397894705455,102223600
3,2018-01-03,40.4197883605957,40.96425916920155,40.356426001755686,40.49019456253724,118071600
4,2018-01-04,40.60752487182617,40.710786792933035,40.384574949985684,40.492527990410416,89738400


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Parar sessão anterior, se necessário
try:
    spark.stop()
except:
    pass

# Criar nova SparkSession com o caminho exato do JAR
spark = SparkSession.builder \
    .appName("PySpark CSV to MySQL") \
    .master("local") \
    .config("spark.jars", "/usr/local/spark-3.4.4-bin-hadoop3/jars/mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

# Caminho do CSV
file_path = "file:///home/hduser/novo/AAPL_AAPL_historical_train_data_spark.csv"

# Leitura do CSV
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Visualizar as primeiras 5 linhas
df.show(5)


+----------+------------------+------------------+------------------+------------------+---------+
|     Price|             Close|              High|               Low|              Open|   Volume|
+----------+------------------+------------------+------------------+------------------+---------+
|    Ticker|              AAPL|              AAPL|              AAPL|              AAPL|     AAPL|
|      Date|              null|              null|              null|              null|     null|
|2018-01-02|40.426815032958984|40.436204433265914| 39.72276041223238| 39.93397894705455|102223600|
|2018-01-03|  40.4197883605957| 40.96425916920155|40.356426001755686| 40.49019456253724|118071600|
|2018-01-04| 40.60752487182617|40.710786792933035|40.384574949985684|40.492527990410416| 89738400|
+----------+------------------+------------------+------------------+------------------+---------+
only showing top 5 rows



In [11]:
# print the data type info
df.printSchema()

root
 |-- Price: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [12]:
df.show(5)

+----------+------------------+------------------+------------------+------------------+---------+
|     Price|             Close|              High|               Low|              Open|   Volume|
+----------+------------------+------------------+------------------+------------------+---------+
|    Ticker|              AAPL|              AAPL|              AAPL|              AAPL|     AAPL|
|      Date|              null|              null|              null|              null|     null|
|2018-01-02|40.426815032958984|40.436204433265914| 39.72276041223238| 39.93397894705455|102223600|
|2018-01-03|  40.4197883605957| 40.96425916920155|40.356426001755686| 40.49019456253724|118071600|
|2018-01-04| 40.60752487182617|40.710786792933035|40.384574949985684|40.492527990410416| 89738400|
+----------+------------------+------------------+------------------+------------------+---------+
only showing top 5 rows



In [13]:
# Remove the first two lines (CRL and null)
df = df.filter(df.Price != "Ticker").filter(df.Price != "Date")

In [14]:
df = df.toDF("Date", "Close", "High", "Low", "Open", "Volume")

In [15]:


df.show(5)



+----------+------------------+------------------+------------------+------------------+---------+
|      Date|             Close|              High|               Low|              Open|   Volume|
+----------+------------------+------------------+------------------+------------------+---------+
|2018-01-02|40.426815032958984|40.436204433265914| 39.72276041223238| 39.93397894705455|102223600|
|2018-01-03|  40.4197883605957| 40.96425916920155|40.356426001755686| 40.49019456253724|118071600|
|2018-01-04| 40.60752487182617|40.710786792933035|40.384574949985684|40.492527990410416| 89738400|
|2018-01-05| 41.06986618041992|41.156698465850205| 40.61223124489687| 40.70375823200148| 94640000|
|2018-01-08| 40.91731262207031|41.213014298775086| 40.81874181549477| 40.91731262207031| 82271200|
+----------+------------------+------------------+------------------+------------------+---------+
only showing top 5 rows



In [16]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [17]:


# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()



+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [18]:
# convert string double
df = df.withColumn("Close", col("Close").cast("double"))

In [19]:
df.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [20]:


df = df.withColumn("High", col("High").cast("double"))
df = df.withColumn("Low", col("Low").cast("double"))
df = df.withColumn("Open", col("Open").cast("double"))
df = df.withColumn("Volume", col("Volume").cast("double"))
df = df.withColumn("Date", col("Date").cast("Date"))



In [21]:


df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)



In [22]:
import mysql.connector
print(mysql.connector.__version__)

9.3.0


In [23]:
df.show(5)

+----------+------------------+------------------+------------------+------------------+----------+
|      Date|             Close|              High|               Low|              Open|    Volume|
+----------+------------------+------------------+------------------+------------------+----------+
|2018-01-02|40.426815032958984|40.436204433265914| 39.72276041223238| 39.93397894705455|1.022236E8|
|2018-01-03|  40.4197883605957| 40.96425916920155|40.356426001755686| 40.49019456253724|1.180716E8|
|2018-01-04| 40.60752487182617|40.710786792933035|40.384574949985684|40.492527990410416| 8.97384E7|
|2018-01-05| 41.06986618041992|41.156698465850205| 40.61223124489687| 40.70375823200148|   9.464E7|
|2018-01-08| 40.91731262207031|41.213014298775086| 40.81874181549477| 40.91731262207031| 8.22712E7|
+----------+------------------+------------------+------------------+------------------+----------+
only showing top 5 rows



In [24]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [25]:
# MySQL connection properties
url = "jdbc:mysql://localhost:3306/companies"  # MySQL URL
properties = {
    "user": "root",
    "password": "password",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [26]:
# Leitura do CSV
#df = spark.read.csv(file_path, header=True, inferSchema=True)

# Adiciona a coluna 'nome' com valor fixo "AAPL"
df_comp_nome = df.withColumn("nome", lit("AAPL"))

# Escreve no banco MySQL
df_comp_nome.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/companies") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "AAPL_historico") \
    .option("user", "root") \
    .option("password", "password") \
    .mode("overwrite") \
    .save()


In [27]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



mySQL

In [ ]:
#pip install pymysql 

In [28]:
import pandas as pd
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='companies'
)

df = pd.read_sql("SELECT * FROM AAPL_historico", conn)
df.to_csv("AAPL_historico_MYsql.csv", index=False)


/tmp/ipykernel_3073/971793236.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM AAPL_historico", conn)


In [29]:
df_AAPL_Mysql= pd.read_csv("AAPL_historico_MYsql.csv")

In [30]:
df_AAPL_Mysql.head()

,Date,Close,High,Low,Open,Volume,nome
0,2018-01-02,40.426815,40.436204,39.722760,39.933979,102223600.0,AAPL
1,2018-01-03,40.419788,40.964259,40.356426,40.490195,118071600.0,AAPL
2,2018-01-04,40.607525,40.710787,40.384575,40.492528,89738400.0,AAPL
3,2018-01-05,41.069866,41.156698,40.612231,40.703758,94640000.0,AAPL
4,2018-01-08,40.917313,41.213014,40.818742,40.917313,82271200.0,AAPL


## cassandra

In [ ]:
pip uninstall cassandra-driver

Found existing installation: cassandra-driver 3.29.2
Uninstalling cassandra-driver-3.29.2:
  Would remove:
    /home/hduser/novo/venv/lib/python3.12/site-packages/cassandra/*
    /home/hduser/novo/venv/lib/python3.12/site-packages/cassandra_driver-3.29.2.dist-info/*
    /home/hduser/novo/venv/lib/python3.12/site-packages/cassandra_driver.libs/libev-a3026d2b.so.4.0.0
Proceed (Y/n)? 

In [3]:
import pandas as pd
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])  # Cassandra local
session = cluster.connect('companies_cassandra')

df = pd.read_csv('AAPL_historico_MYsql.csv')

for _, row in df.iterrows():
    session.execute(
        """
        INSERT INTO aapl_historico (date, close, high, low, open, volume, nome)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (row['Date'], row['Close'], row['High'], row['Low'], row['Open'], row['Volume'], row['nome'])
    )

NoHostAvailable: ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})